# 📊 HubSpot Pipeline Health Analyzer
## Data-Driven Pipeline Management for K-8 Education Sales
---
**Purpose:** Connect to HubSpot API → analyze pipeline health → surface insights  
that help a sales rep prioritize deals and forecast accurately.

**Why this exists:**
Most reps manage their pipeline by gut. This notebook applies data science  
to your own pipeline data to answer: which deals are likely to close,  
which are at risk, and where to focus your next 48 hours.


## 🔧 Setup + HubSpot Connection

In [ ]:
# Fix sys.path to include src directory
import os
import sys
_nb_dir = os.path.dirname(os.path.abspath('__file__'))
_repo_root = os.path.abspath(os.path.join(_nb_dir, '..'))
sys.path.insert(0, os.path.join(_repo_root, 'src'))

import os, json, requests, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()
HUBSPOT_API_KEY = os.getenv("HUBSPOT_API_KEY")
BASE_URL = "https://api.hubapi.com"
HEADERS = {"Authorization": f"Bearer {HUBSPOT_API_KEY}", "Content-Type": "application/json"}

print(f"HubSpot connection configured.")
print(f"API key loaded: {bool(HUBSPOT_API_KEY)}")
print("Note: Run cells below only with a valid HUBSPOT_API_KEY in your .env file")


## 📥 Step 1: Pull Pipeline Data

In [ ]:
# ============================================================
# HUBSPOT API: PULL ALL DEALS
# Docs: https://developers.hubspot.com/docs/api/crm/deals
# TODO (Jules): Test with real HubSpot sandbox
# ============================================================

def get_all_deals(api_key):
    url = f"{BASE_URL}/crm/v3/objects/deals"
    params = {
        "limit": 100,
        "properties": [
            "dealname", "amount", "dealstage", "closedate",
            "hs_deal_stage_probability", "createdate",
            "num_contacted_notes", "hs_lastmodifieddate",
            "hubspot_owner_id"
        ],
    }
    headers = {"Authorization": f"Bearer {api_key}"}
    all_deals = []
    after = None

    while True:
        if after:
            params["after"] = after
        try:
            resp = requests.get(url, headers=headers, params=params)
            data = resp.json()
            all_deals.extend(data.get("results", []))
            after = data.get("paging", {}).get("next", {}).get("after")
            if not after:
                break
            time.sleep(0.2)
        except Exception as e:
            print(f"API error: {e}")
            break

    return all_deals


# --- MOCK DATA for demo (replace with get_all_deals(HUBSPOT_API_KEY)) ---
np.random.seed(42)
n_deals = 25
stages = ["discovery", "evaluation", "proposal_sent", "negotiation", "closed_won", "closed_lost"]
stage_probs = [0.1, 0.25, 0.4, 0.65, 1.0, 0.0]

deals_df = pd.DataFrame({
    "deal_name": [f"District_{i:02d} — Literacy PD Pilot" for i in range(n_deals)],
    "amount": np.random.choice([50000, 75000, 100000, 150000, 200000], n_deals),
    "stage": np.random.choice(stages, n_deals, p=[0.25, 0.25, 0.2, 0.1, 0.1, 0.1]),
    "close_date": [datetime.now() + timedelta(days=int(d)) for d in np.random.randint(7, 180, n_deals)],
    "days_since_last_activity": np.random.randint(0, 45, n_deals),
    "contact_count": np.random.randint(1, 15, n_deals),
    "created_date": [datetime.now() - timedelta(days=int(d)) for d in np.random.randint(7, 120, n_deals)],
})

stage_prob_map = dict(zip(stages, stage_probs))
deals_df["stage_probability"] = deals_df["stage"].map(stage_prob_map)
deals_df["weighted_value"] = deals_df["amount"] * deals_df["stage_probability"]

print(f"Pipeline loaded: {len(deals_df)} deals")
print(f"Total pipeline value: ${deals_df['amount'].sum():,.0f}")
print(f"Weighted pipeline:    ${deals_df['weighted_value'].sum():,.0f}")
print(deals_df[["deal_name","amount","stage","stage_probability","close_date"]].head(10).to_string(index=False))


## ⚠️ Step 2: At-Risk Deal Detection

In [ ]:
# ============================================================
# AT-RISK SIGNAL DETECTION
# Flags deals that need immediate attention
# ============================================================

def flag_at_risk(row):
    risks = []
    if row["days_since_last_activity"] > 14 and row["stage"] not in ["closed_won","closed_lost"]:
        risks.append(f"No activity in {row['days_since_last_activity']} days")
    if row["close_date"] < datetime.now() + timedelta(days=14) and row["stage"] not in ["closed_won","closed_lost","negotiation"]:
        risks.append("Close date <14 days but not in negotiation")
    if row["contact_count"] < 3 and row["stage"] in ["evaluation","proposal_sent"]:
        risks.append("Low contact count for this stage")
    return "; ".join(risks) if risks else "Healthy"

deals_df["risk_flags"] = deals_df.apply(flag_at_risk, axis=1)
at_risk = deals_df[deals_df["risk_flags"] != "Healthy"]

print("AT-RISK DEALS — ACTION REQUIRED TODAY")
print("=" * 60)
for _, row in at_risk.iterrows():
    print(f"
  {row['deal_name']}")
    print(f"  Stage: {row['stage']} | Value: ${row['amount']:,}")
    print(f"  Risk: {row['risk_flags']}")

print(f"
Summary: {len(at_risk)} of {len(deals_df)} deals at risk")
print(f"At-risk pipeline value: ${at_risk['weighted_value'].sum():,.0f}")


In [ ]:
# Pipeline Visualization
BRAND = {"primary": "#2E4057", "secondary": "#048A81", "accent": "#F18F01", "danger": "#C73E1D"}

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle("HubSpot Pipeline Health Dashboard", fontsize=14, fontweight="bold", color=BRAND["primary"])

# Pipeline by Stage
stage_summary = deals_df.groupby("stage").agg(
    count=("deal_name","count"),
    total_value=("amount","sum"),
    weighted_value=("weighted_value","sum")
).reindex(stages).fillna(0)

axes[0].bar(stage_summary.index, stage_summary["total_value"]/1000,
            color=[BRAND["secondary"] if s != "closed_lost" else BRAND["danger"] for s in stage_summary.index],
            alpha=0.85)
axes[0].set_title("Pipeline Value by Stage ($K)")
axes[0].set_xticklabels(stage_summary.index, rotation=25, ha="right")
axes[0].set_ylabel("Total Value ($K)")

# Days Since Last Activity
colors = [BRAND["danger"] if d > 14 else BRAND["secondary"] for d in deals_df["days_since_last_activity"]]
axes[1].scatter(range(len(deals_df)), sorted(deals_df["days_since_last_activity"]),
                c=colors, alpha=0.8, s=100)
axes[1].axhline(y=14, color="red", linestyle="--", label="14-day at-risk threshold")
axes[1].set_title("Days Since Last Activity (Red = At Risk)")
axes[1].set_xlabel("Deal Index")
axes[1].set_ylabel("Days Since Last Activity")
axes[1].legend()

plt.tight_layout()
plt.savefig("pipeline_health_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()
